# Lab 3: Building Agents with memory 

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [ ]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [1]:
from helper import nb_print

In [2]:
from letta import create_client 

client = create_client() 

Saved Config:  /home/jovyan/.letta/config
📖 Letta configuration file updated!
🧠 model	-> gpt-4
🖥️  endpoint	-> http://jupyter-api-proxy.internal.dlai/rev-proxy/letta
Saved Config:  /home/jovyan/.letta/config
Saved Config:  /home/jovyan/.letta/config


In [3]:
from letta.schemas.llm_config import LLMConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

## Section 1: Creating a simple agent with memory 

### Creating an agent 

In [4]:
agent_name = "simple_agent"

In [5]:
# this is not in the video. It deletes the agent if you are running this a 2nd time.
if client.get_agent_id(agent_name): 
    client.delete_agent(client.get_agent_id(agent_name))

In [6]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a helpful assistant that loves emojis"
    )
)

In [7]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)

In [8]:
response.usage

LettaUsageStatistics(completion_tokens=73, prompt_tokens=2124, total_tokens=2197, step_count=1)

In [9]:
nb_print(response.messages)

### Understanding agent state 

In [10]:
print(agent_state.system)

You are MemGPT, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
Hi

In [11]:
agent_state.tools

['send_message',
 'pause_heartbeats',
 'conversation_search',
 'conversation_search_date',
 'archival_memory_insert',
 'archival_memory_search',
 'core_memory_append',
 'core_memory_replace']

### Viewing an agent's memory

In [12]:
agent_state.memory

Memory(memory={'persona': Block(value='You are a helpful assistant that loves emojis', limit=2000, name='persona', template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-d14bbf36-0e7a-4e24-bcc2-36f9d844d46d'), 'human': Block(value='My name is Sarah', limit=2000, name='human', template=False, label='human', description=None, metadata_={}, user_id=None, id='block-fbc9a646-6d67-403a-a3a8-2a2179cc0e8f')}, prompt_template='{% for block in memory.values() %}<{{ block.name }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.name }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [13]:
client.get_archival_memory_summary(agent_state.id)

ArchivalMemorySummary(size=0)

In [14]:
client.get_recall_memory_summary(agent_state.id)

RecallMemorySummary(size=9)

In [15]:
client.get_messages(agent_state.id)[0]

Message(id='message-b9e3f4ff-a2b1-47d0-a810-d01bd0df0f1e', role=<MessageRole.tool: 'tool'>, text='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-12-11 05:18:22 PM UTC+0000"\n}', user_id='user-724c8a36-5f56-4b10-b17d-9713cf7c8ea2', agent_id='agent-23c7fd1e-0414-435c-bce6-e06f5f29ef58', model='gpt-4', name='send_message', created_at=datetime.datetime(2024, 12, 11, 17, 18, 22, 95896), tool_calls=None, tool_call_id='9df407df-ec34-4e87-ae4c-1126c')

## Section 2: Understanding core memory 

### Memories about the human 

In [17]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Bob", 
    role = "user"
) 
nb_print(response.messages)

### Memories about the agent

In [18]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate", 
    role = "user"
) 
nb_print(response.messages)

In [19]:
client.get_core_memory(agent_state.id).get_block('persona')

Block(value='You are a helpful assistant that avoids using emojis', limit=2000, name='persona', template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-d14bbf36-0e7a-4e24-bcc2-36f9d844d46d')

## Section 3: Understanding archival memory

In [20]:
client.get_archival_memory(agent_state.id)

[]

In [21]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'bob loves cats' to archival", 
    role = "user"
) 
nb_print(response.messages)

In [22]:
client.get_archival_memory(agent_state.id)[0].text

'Bob loves cats'

In [23]:
passage = client.insert_archival_memory(
    agent_state.id, 
    "Bob loves Boston Terriers"
)

In [24]:
passage

[Passage(user_id='user-724c8a36-5f56-4b10-b17d-9713cf7c8ea2', agent_id='agent-23c7fd1e-0414-435c-bce6-e06f5f29ef58', source_id=None, doc_id=None, metadata_={}, id='passage-123898f8-c16d-4a78-9443-c2e90be1a216', text='Bob loves Boston Terriers', embedding=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='http://jupyter-api-proxy.internal.dlai/rev-proxy/letta', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), created_at=datetime.datetime(2024, 12, 11, 17, 27, 21))]

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
nb_print(response.messages)